<a href="https://colab.research.google.com/github/KshitijMayank/Twitter_Scraping/blob/master/Web_Scrapping(Twitter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Personal Acces OAuth Details**

In [0]:
# !pip install twitter 
# pip install python-twitter
# pip install tweepy

Automating the code to fetch tweets from twitter based on the limits on API calls. This will fetch around ~15000 rows over the added parameters.

**Setting up Connection**

In [0]:
# Importing tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
# import preprocessor as p
# import os
import time


#getting our access Credentials...
consumer_key='wGDrJqIQaGzEW4EN36ypSWCaY'
consumer_secret='MOZ2jD2ryo4dJoDx6WsheFPVDMTsNwghpdc8q3D72tBg0FHpHp'
access_token_key='1191970425095565313-EcKOnnwGrJ5WTY1TqpEd7vFq1g0hoi'
access_token_secret='jF1LtDylkr1TrUAnaG6A6A1lXDGxxMQlUsOE9e1b6qQET'

# Authenticating our python script
# initialize api instance
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token_key,access_token_secret)
api = tweepy.API(auth)
# test authentication
try:
    api.verify_credentials()
    print("All good to go")
except:
    print("Error during authentication")

# Automating the process to get tweets for the most popular election hashtags...


def scraptweets(search_words, date_since, numTweets, numRuns):

    ## Arguments:
    # search_words -> define a string of keywords for this function to extract
    # date_since -> define a date from which to start extracting the tweets 
    # numTweets -> number of tweets to extract per run
    # numRun -> number of runs to perform in this program - API calls are limited to once every 15 mins, so each run will be 15 mins apart.
    ##
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    # Define a for-loop to generate tweets at regular intervals
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)

        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]

        # Obtain the following info (methods to call them out):
            # user.screen_name - twitter handle
            # user.description - description of account
            # user.location - where is he tweeting from
            # user.friends_count - no. of other users that user is following (following)
            # user.followers_count - no. of other users who are following this user (followers)
            # user.statuses_count - total tweets by user
            # user.created_at - when the user account was created
            # created_at - when the tweet was created
            # retweet_count - no. of retweets
            # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
            # retweeted_status.full_text - full text of the tweet
            # tweet.entities['hashtags'] - hashtags in the tweet

        # Begin scraping the tweets individually:
        noTweets = 0

        for tweet in tweet_list:

            # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text

            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

            # Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet

            # increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round(end_run-start_run, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
        print('time take for {} run to complete is {}'.format(i, duration_run))
        
        time.sleep(900) #15 minute sleep time

        
    # Once all runs have completed, save them to a single csv file:    
    # Obtain timestamp in a readable format:
    from datetime import datetime
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

    # Define working path and filename
    # path = os.getcwd()
    filename = to_csv_timestamp + '_Elections(set7).csv'

    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    print('Scraping has completed!')


#Our Lexicons for the program


search_words = "@BernieSanders OR @SenSanders OR #BernieSanders OR #Bernie2020 OR #BERNIE2020 OR #NotMeUs OR #MedicareForAll OR #FeelTheBern OR @JoeBiden OR  @TeamJoe OR #TeamJoe OR #Biden OR #Biden2020 OR #JoeBiden OR #TimesUpJoe OR #CNNTownHall OR #TimesUpBiden"
date_since = "2020-15-02"
numTweets = 2500
numRuns = 6
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

All good to go
no. of tweets scraped for run 0 is 0
time take for 0 run to complete is 0.26
no. of tweets scraped for run 1 is 0
time take for 1 run to complete is 0.17
no. of tweets scraped for run 2 is 0
time take for 2 run to complete is 0.21
no. of tweets scraped for run 3 is 0
time take for 3 run to complete is 0.24
no. of tweets scraped for run 4 is 0
time take for 4 run to complete is 0.17
no. of tweets scraped for run 5 is 0
time take for 5 run to complete is 0.21
Scraping has completed!


### **Lexicons**

In [0]:

# set5 ='#ForThePeople OR #2020Elections OR #hilory OR #Election2020 OR #Elections2020 OR #trump OR #makeamericagreatagain OR #republican OR #Democrats2020 OR #vote OR #trump OR  #voteblue OR #maga OR #impeachtrump OR #resist OR #donaldtrump OR #bernie OR #berniesanders2020'
# set2='#america OR #joebiden OR #impeachment OR #democrat OR #electionday OR #berniesanders OR #democraticprimary OR #biden OR #pedovores OR #resist OR #mayorpete OR #andrewyang OR #walkaway OR #politicalmemes OR #impeachtrump OR #pedogate OR #liberals OR #gop OR #peteforamerica'
# set1="#2020Elections OR #JoeBiden OR #Bernie2020 OR #Election2020 OR #Elections2020 OR #NotMeUs OR #makeamericagreatagain OR #electionday OR #berniesanders OR #joebiden OR #Democrats2020 OR #election OR #democrats OR #vote OR #trump OR @GeraldoRivera OR @realDonaldTrump OR @JoeBiden OR #CoronavirusPandemic OR @BernieSanders"
# set3="#liberal OR #progressive OR #politics OR #resistance OR #yanggang OR #trumpsucks OR #america OR #kag OR #usa OR #democrat OR #votebluenomatterwho OR #treason OR #petebuttigieg OR #qanon OR #votedemocrat OR #blackvotesmatter OR #getoutthevote"
# set4="#votebluetoendthisnightmare OR #bhfyp OR #america OR #joebiden OR #impeachment OR #democrat OR #electionday OR #berniesanders OR #democraticprimary OR #politics OR #pedovores OR #resist OR #mayorpete OR #andrewyang OR #walkaway OR #politicalmemes OR #impeachtrump OR #pedogate OR #liberals OR #gop OR #peteforamerica"


set7 ="@BernieSanders OR @SenSanders OR #BernieSanders OR #Bernie2020 OR #BERNIE2020 OR #NotMeUs OR #MedicareForAll OR #FeelTheBern OR @JoeBiden OR  @TeamJoe OR #TeamJoe OR #Biden OR #Biden2020 OR #JoeBiden OR #TimesUpJoe OR #CNNTownHall OR #TimesUpBiden"





#america OR OR OR #joebiden OR OR OR #impeachment OR OR OR #democrat OR OR OR #electionday OR OR OR #berniesanders OR OR OR #democraticprimary OR OR OR #biden OR OR OR #pedovores OR OR OR #resist OR OR OR #mayorpete OR OR OR #andrewyang OR OR OR #walkaway OR OR OR #politicalmemes OR OR OR #impeachtrump OR OR OR #pedogate OR OR OR #liberals OR OR OR #gop OR OR OR #peteforamerica
